In [ ]:
%run utilspro.py

In [ ]:
# Loading the dataset
data = pd.read_csv(dataset_path)

# Displaying the first few rows of the dataset
data.head()

### Data Cleaning Steps:
- Handle placeholder values (?).
- Convert date columns to the proper datetime format.
- Address the inconsistency with incidents marked as active but having a closed_at date.
- Handle any other anomalies and inconsistencies identified during the EDA.

#### Handling placeholder values (?)
We previously identified several columns containing the placeholder value ?. We'll replace these placeholders with appropriate NaN (null) values, which will allow us to handle them more effectively.

In [ ]:
# Replacing '?' with NaN
data.replace('?', pd.NA, inplace=True)

# Checking the number of missing values in each column after replacement
missing_values = data.isna().sum()

missing_values[missing_values > 0]

### Model Based imputation

Given that the KNN imputer may generate new labels not seen during the initial label encoding, we need to ensure that, post-imputation, only the known labels are used for inverse transformation.

strategy:

Using the KNN imputer on the data as before.
Post-imputation, for the columns that were label-encoded, clip any values that lie outside the range [0, number of classes for that column - 1].
Use inverse transformation on these clipped values.

In [ ]:
missing_values_cleaned_updated=missing_values[missing_values > 0]

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import KNNImputer

# Selecting columns with missing values
columns_with_missing = missing_values_cleaned_updated.index.tolist()

# Creating a subset of data with these columns
data_missing = data.copy()[columns_with_missing]

# Label Encoding for categorical variables
label_encoders = {}
for col in columns_with_missing:
    if data_missing[col].dtype == 'object':
        le = LabelEncoder()
        data_missing[col] = le.fit_transform(data_missing[col].astype(str))
        label_encoders[col] = le

# Scaling the data for KNN
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_missing)

# Scaling the data for KNN
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_missing)

# KNN Imputer initialization and imputation
knn_imputer = KNNImputer(n_neighbors=5)
data_imputed = knn_imputer.fit_transform(data_scaled)

# Convert imputed data back to dataframe
data_imputed_df = pd.DataFrame(data_imputed, columns=columns_with_missing)

# Clipping values for label encoded columns to ensure they lie within the known labels range
for col, le in label_encoders.items():
    max_label = len(le.classes_) - 1
    data_imputed_df[col] = data_imputed_df[col].clip(0, max_label).astype(int)

    # Inverse transform for label encoded columns
    data_imputed_df[col] = le.inverse_transform(data_imputed_df[col])

# Checking if missing values are imputed and if any unknown labels were introduced
missing_after_imputation = data_imputed_df.isna().sum()

missing_after_imputation
#check the number of columns before and after imputation
print('Number of columns before imputation: ', data.shape[1])
print('Number of columns after imputation: ', data_imputed_df.shape[1])

# Replacing the columns in the original dataset with the imputed versions
data_cleaning = data.copy()
for col in columns_with_missing:
    data_cleaning[col] = data_imputed_df[col]

# Checking the number of columns after the replacement
num_columns_after_replacement = data_cleaning.shape[1]

print('Number of columns after replacement', num_columns_after_replacement)

data_cleaning.isna().sum()


In [ ]:
data_cleaning.columns

In [ ]:
# Converting date columns to datetime format
date_columns = ['opened_at', 'sys_created_at', 'resolved_at', 'closed_at']
for column in date_columns:
    data_cleaning[column] = data_cleaning[column].apply(robust_date_parser)

# Checking the datatypes of the columns after conversion
data_cleaning[date_columns].dtypes


Let's address the anomalies and inconsistencies we identified during our exploratory data analysis (EDA):

- Incidents with closed_at dates but marked as active: 
We found 116,726 such incidents. This is inconsistent since active incidents should not have a closure date.
- Potential Outliers:
Reassignment Count: Some incidents have been reassigned more than 20 times.
- Reopen Count: 
Some incidents have been reopened multiple times.
- Sys Mod Count: 
Some incidents have more than 40 system modifications.

### Addressing Anomalies:
- Incidents with closed_at dates but marked as active:
### Our solution: Set these incidents as inactive (active = False).

- Potential Outliers:
For each of the columns (Reassignment Count, Reopen Count, Sys Mod Count), we can:
a. Cap the values at a certain threshold based on domain knowledge or statistical measures (like the 95th percentile).
b. Investigate further to understand the reasons for such high values.
c. Leave them as they are if they represent genuine scenarios.

In [ ]:
# Setting incidents with 'closed_at' dates but marked as 'active' to inactive
data_cleaning.loc[(data_cleaning['active'] == True) & (data_cleaning['closed_at'].notna()), 'active'] = False

# Checking the number of incidents that are still marked as 'active' but have a 'closed_at' date
active_with_closed_date = data_cleaning[(data_cleaning['active'] == True) & (data_cleaning['closed_at'].notna())].shape[0]

active_with_closed_date

### Potential Outliers:

For columns : Reassignment Count, Reopen Count, and Sys Mod Count, we can cap values beyond the 0.95 quantile threshold as follows:

In [ ]:
columns_list = ['reassignment_count', 'reopen_count', 'sys_mod_count']
for column in columns_list:
    threshold = data_cleaning[column].quantile(0.95)
    data_cleaning[column] = data_cleaning[column].apply(lambda x: threshold if x > threshold else x)

## Feature engineering
- Date Features: Extract relevant information from date columns.
Day of the week, hour, and month from the opened_at column.
Time taken to resolve an incident (difference between closed_at and opened_at).

- Categorical Features: Convert categorical variables into a format suitable for machine learning models.
One-hot encode categorical columns like incident_state, contact_type, and priority.
For high cardinality categorical columns, consider using target encoding or other encoding techniques.

- Text Features: If there are textual descriptions or notes in the dataset, derive features from them.
Text length, sentiment analysis, or even more advanced techniques like TF-IDF or embeddings (this would depend on the nature and quality of the text data).

- Interaction Features: Create interaction terms between relevant features, which can sometimes capture patterns that individual features might miss.

- Normalization: Depending on the model we decide to use later, we might need to normalize or standardize some numerical features.

Let's start by extracting features from the date columns, specifically from the opened_at column. We'll derive the day of the week, hour, and month from it.

In [ ]:
# Extracting features from the 'opened_at' column
data_cleaning['opened_day_of_week'] = data_cleaning['opened_at'].dt.dayofweek
data_cleaning['opened_hour'] = data_cleaning['opened_at'].dt.hour
data_cleaning['opened_month'] = data_cleaning['opened_at'].dt.month

# Calculating the resolution time in hours (if it hasn't been calculated already)
if 'resolution_time' not in data_cleaning.columns:
    data_cleaning['resolution_time'] = (data_cleaning['closed_at'] - data_cleaning['opened_at']).dt.total_seconds() / (60 * 60)

# Displaying the first few rows with the new features
data_cleaning[['opened_at', 'opened_day_of_week', 'opened_hour', 'opened_month', 'resolution_time']].head()


In [ ]:
# Calculating the resolution duration in hours
data_cleaning['resolution_duration'] = (data_cleaning['closed_at'] - data_cleaning['opened_at']).dt.total_seconds() / 3600

# Displaying the first few rows with the 'resolution_duration' feature
data_cleaning[['opened_at', 'closed_at', 'resolution_duration']].head()

#### Checking for duplicated entries

In [ ]:
# Check for duplicates
data_cleaning.duplicated().sum()

In [ ]:
# display the duplicated rows and make a list out  of the 'number' column values
duplicated_rows = data_cleaning[data_cleaning.duplicated(keep=False)].sort_values(by='number')
duplicated_rows['number'].tolist()

In [ ]:
# make a dataframe with the duplicated rows stating the number of duplicates per unique number value
duplicated_rows_count = duplicated_rows.groupby('number').size().reset_index(name='count')
# plot the most numeros duplicated rows count in a descending order
duplicated_rows_count.sort_values(by='count', ascending=False).head(10).plot.bar(x='number', y='count', figsize=(10, 6))
#label the plot
plt.title('Number of duplicates per unique number value')
plt.xlabel('Number')
plt.ylabel('Count')
plt.tight_layout()
plt.show()



In [ ]:
#check the duplicates for number 'INC0019396' order by duplicated sequence  
duplicated_rows[duplicated_rows['number'] == 'INC0019396'].sort_values(by='sys_created_at')

In [ ]:
# drop the duplicates
data_cleaning.drop_duplicates(inplace=True)
# check the number of rows after dropping the duplicates
data_cleaning.shape[0]

In [ ]:
#drop the number column
data_cleaning.drop('number', axis=1, inplace=True)
# check the aspect of all categorical columns uniques values count
data_cleaning.select_dtypes('object').nunique()

In [ ]:
data_cleaning.columns

In [ ]:
%run utilspro.py
encoder = SmartEncoder()
# Encoding the categorical columns with the SmartEncoder object the target column is 'acti
encoded_df = encoder.fit_transform(data_cleaning, target_column='active')


In [ ]:
# Displaying the first few rows of the encoded dataframe
encoded_df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Exclude datetime columns and target column
X = encoded_df.drop(['opened_at', 'sys_created_at', 'resolved_at', 'closed_at', 'active'], axis=1)

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print(f"Original number of features: {X_scaled.shape[1]}")
print(f"Reduced number of features: {X_pca.shape[1]}")

In [ ]:
# chenck the number of components
pca.n_components_

In [ ]:
# use variance ratio to check the explained variance of the components
pca.explained_variance_ratio_

In [ ]:
# bypass_columns = ['number', 'opened_at', 'sys_created_at', 'resolved_at', 'closed_at', 'resolution_time', 'resolution_duration']

In [ ]:
# build a longitudinal correlation funnel for the target column 'active'
# write a function to plot the correlation funnel

def plot_correlation_funnel(data, target_column, figsize=(10, 6)):
    """
    Plots a correlation funnel for the target column of a dataframe
    """
    # Calculate the correlation coefficients
    corr = data.corr()[target_column].sort_values(ascending=False)

    # Exclude the target column
    corr = corr.drop(target_column)

    # Calculate the number of features
    num_features = corr.shape[0]

def correlation_funnel(df, target_column, corr_threshold=0.1, multicollinearity_threshold=0.8):
    # Step 1 & 2: Calculate correlation with target and filter
    corr_with_target = df.drop(columns=target_column).apply(lambda x: x.corr(df[target_column]))
    significant_features = corr_with_target[corr_with_target.abs() > corr_threshold].index.tolist()
    
    # Step 3: Check for multicollinearity
    corr_matrix = df[significant_features].corr().abs()
    
    # Create a mask for the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Find features with correlation greater than the threshold
    to_drop = [column for column in upper.columns if any(upper[column] > multicollinearity_threshold)]
    
    # Drop features 
    significant_features = [feat for feat in significant_features if feat not in to_drop]
    
    return significant_features

# Example usage:
# selected_features = correlation_funnel(df, 'target_column_name')

    # Plot the funnel
    plt.figure(figsize=figsize)
    plt.barh(range(num_features), corr)
    plt.yticks(np.arange(num_features), corr.index.tolist())
    plt.title('Correlation Funnel')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()

%run utilspro.py
#code active's True as 1 and False as 0
data_cleaning['active'] = data_cleaning['active'].astype(int)
# Plot the correlation funnel for the target column 'active' using numerical columns only
plot_correlation_funnel(data_cleaning.select_dtypes('number'), 'active')



